# Connecting to Hardware

**SUMMARY**: *In this lab, you'll learn how to setup and connect to your ChipWhisperer hardware. We'll also cover how to build firmware for your target microcontroller, how to capture power traces, and how to communicate with target devices.*

*All the API calls we'll be using here are documented on the [ChipWhisperer readthedocs page](https://chipwhisperer.readthedocs.io/en/latest/api.html), so feel free to open it in another tab and follow along there as well.*

**LEARNING OUTCOMES:**
* Setting up your ChipWhisperer Hardware
* Using the ChipWhisperer Python API to connect to your hardware
* Communication with the target
* Capturing a power trace

## Prerequisites

Hold up - before continuing, ensure you have done the following:

* ☑ Validated your setup using the [ChipWhisperer Setup Test](./ChipWhisperer%20Setup%20Test.ipynb) notebook.
* ☑ Run through the Jupyter introduction.

## Physical Setup

### ChipWhisperer-Nano

The ChipWhisperer-Nano is a single-board device. It includes the capture hardware, along with a built in STM32F0 target. To use this target, simply plug in the device, then go get a drink to reward yourself:

<img src="img/cwnano.jpg" alt="CW-Nano Plugged In" width=400>

### ChipWhisperer-Lite (Single Board)

The ChipWhisperer-Lite is a single-board device. It includes the capture hardware, along with a built in STM32F3 or XMEGA target. To use this target, simply plug in the device:

<img src="img/cwlite_plugged.jpg" alt="CW-Lite Plugged In" width=400>


### ChipWhisperer-Lite (2-Part Version)

The ChipWhisperer-Lite 2-Part version includes the ChipWhisperer-Capture along with an XMEGA target. You need to attach the SMA & IDE cables, then plug in the USB cable:

<img src="img/cw_2part.jpg" alt="CW-Lite 2-Part Connected" width=500>

### ChipWhisperer-Lite (Capture + UFO), Includes SCAPACK-L1/SCAPACK-L2

This package uses a ChipWhisperer-Lite Capture board, along with a CW308 UFO target board. You'll need to attach the CW308 board to your ChipWhisperer-Lite Capture, and plug in your choosen target board. The suggested target for most tutorials is the `CW308_STM32F3` - you can tell this target by checking the part number on the chip, look for a `STM32F303`.

Here is an overview photo of the target plugged in:

<img src="img/cwcapture_ufo.jpg" alt="CW-Lite Plugged Into UFO" width=600>

See Section below on UFO Target Settings.

### ChipWhisperer-Pro

This package uses a ChipWhisperer-Pro Capture device, along with a CW308 UFO target board. You'll need to attach the CW308 board to your ChipWhisperer-Pro Capture, and plug in your choosen target board. The suggested target for most tutorials is the `CW308_STM32F3` - you can tell this target by checking the part number on the chip, look for a `STM32F303`.

Here is an overview photo of the business end of the CW-Pro plugged into the UFO board. You'll need to also plug in the USB-A cable, and possibly the DC power jack (2.1mm DC power jack, 5V comes from either USB or wall adapter). Earlier hardware revisions always needed the DC power jack, later hardware revisions will use USB power (but high-power targets still need extra juice).

<img src="img/cwpro_ufo.jpg" alt="CW-Pro Plugged In" width=500>

See Section below on UFO Target Settings.

### UFO Target Settings

The UFO Board comes by default with working settings for most targets. A summary of the default jumper settings is included below for you, see the product documentation for more details.

<img src="img/cwufo_stm32f3.jpg" alt="UFO Board" width=500>

#### UFO Default Settings

* J1  - 3.3V (CW3.3V)
* J3  - HS2/OUT
* J14 - FILT/FILT_LP/FILT_HP
* J16 - No connection

* S1 -  ON

* 3.3V SRC - J1/CW
* VADJ SRC - 3.3V
* 1.2V, 1.8V, 2.5V, 3.3V, LDO SRC - J1/CW
* 5V SRC - J1/CW

### CW-Lite single board: XMEGA vs ARM

If you are unsure which single-part version you have, the following shows both boards. Note the only difference is the target section on the right.

<img src="img/cwlitearm_vs_cwlitexmega.jpg" alt="CW-Lite XMEGA vs Arm Board" width=500>


## Connecting to ChipWhisperer

Now that your hardware is all setup, we can now learn how to connect to it. We can connect to the ChipWhisperer with:

In [1]:
import chipwhisperer as cw
scope = cw.scope()

By default, ChipWhisperer will try to autodetect the type of device your're running (CWLite/CW1200 or CWNano), see API documentation for manually specifying the scope type. If you have multiple ChipWhisperer devices connected, you'll need to specify the serial number of the device you want to connect to:

```python
scope = cw.scope(sn='<some long string of numbers>')
```

For more information, see the API section on readthedocs.

Connecting to the target device is simple as well:

In [2]:
target = cw.target(scope, cw.targets.SimpleSerial) #cw.targets.SimpleSerial can be omitted

We'll only be discussing the default target type, which is SimpleSerial. Other targets, like the CW305, will be covered in hardware specific demos. 

Some sane default settings can be set using:

In [3]:
scope

cwlite Device
sn         = 50203220313038543030393131313039
fw_version = 
    major = 0
    minor = 51
    debug = 0
gain = 
    mode = low
    gain = 0
    db   = 5.5
adc = 
    state          = False
    basic_mode     = low
    timeout        = 2
    offset         = 0
    presamples     = 0
    samples        = 24400
    decimate       = 1
    trig_count     = 9497212
    fifo_fill_mode = normal
clock = 
    adc_src       = clkgen_x1
    adc_phase     = 0
    adc_freq      = 5264259
    adc_rate      = 5264259.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 1
    clkgen_freq   = 192000000.0
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1         = serial_tx
    tio2         = serial_rx
    tio3         = high_z
    tio4         = high_z
    pdid         = high_z
    pdic         = high_z
    nrst         = high_z
 

In [4]:
scope.default_setup()
scope

cwlite Device
sn         = 50203220313038543030393131313039
fw_version = 
    major = 0
    minor = 51
    debug = 0
gain = 
    mode = high
    gain = 30
    db   = 24.8359375
adc = 
    state          = False
    basic_mode     = rising_edge
    timeout        = 2
    offset         = 0
    presamples     = 0
    samples        = 5000
    decimate       = 1
    trig_count     = 16143060
    fifo_fill_mode = normal
clock = 
    adc_src       = clkgen_x1
    adc_phase     = 0
    adc_freq      = 96000000
    adc_rate      = 96000000.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 26
    clkgen_freq   = 7384615.384615385
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1         = serial_rx
    tio2         = serial_tx
    tio3         = high_z
    tio4         = high_z
    pdid         = high_z
    pdic         = high_z
 

which from its [documentation](https://chipwhisperer.readthedocs.io/en/latest/api.html#chipwhisperer.capture.scopes.OpenADC.OpenADC.default_setup) you can see does the following for the CWLite/CW1200:

* Sets the scope gain to 45dB
* Sets the scope to capture 5000 samples
* Sets the scope offset to 0 (aka it will begin capturing as soon as it is triggered)
* Sets the scope trigger to rising edge
* Outputs a 7.37MHz clock to the target on HS2
* Clocks the scope ADC at 4\*7.37MHz. Note that this is *synchronous* to the target clock on HS2
* Assigns GPIO1 as serial RX
* Assigns GPIO2 as serial TX

And that's it! Your ChipWhisperer is now setup and ready to attack a target. 

**NOTE: You'll need to disconnect the scope/target before connecting again, like you would in another notebook. This can be done with `scope.dis()` and `target.dis()`**

## Building and Uploading Firmware

The next step in attacking a target is to get some firmware built and uploaded onto it. Most firmware for most ChipWhisperer targets can be built using our build system, provided you have the correct compiler installed (see https://chipwhisperer.readthedocs.io/en/latest/prerequisites.html#prerequisites for info about compilers).

Firmware must be built on the command line. Luckily, thanks to Jupyter, we can run a command within a notebook as follows:

In [5]:
%%bash
cd ../hardware/victims/firmware/simpleserial-base/
make PLATFORM=CWLITEXMEGA CRYPTO_TARGET=NONE

SS_VER set to SS_VER_1_1
rm -f -- simpleserial-base-CWLITEXMEGA.hex
rm -f -- simpleserial-base-CWLITEXMEGA.eep
rm -f -- simpleserial-base-CWLITEXMEGA.cof
rm -f -- simpleserial-base-CWLITEXMEGA.elf
rm -f -- simpleserial-base-CWLITEXMEGA.map
rm -f -- simpleserial-base-CWLITEXMEGA.sym
rm -f -- simpleserial-base-CWLITEXMEGA.lss
rm -f -- objdir-CWLITEXMEGA/*.o
rm -f -- objdir-CWLITEXMEGA/*.lst
rm -f -- simpleserial-base.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s
rm -f -- simpleserial-base.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d
rm -f -- simpleserial-base.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i
.
Welcome to another exciting ChipWhisperer target build!!
avr-gcc (GCC) 5.4.0
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: simpleserial-base.c
a

You should see a big list of `PLATFORM`s to build for. We left the `PLATFORM` blank in the command above, so the build system instead printed a list of supported platforms. Fill in your platform, rerun the build command, and your firmware should be successfully built.

Continuing on, there's two possible ways to upload firmware to your target:

1. ChipWhisperer has built in support for XMEGA, STM32F*, and AVR bootloaders. These can be used as follows

In [6]:
cw.program_target(scope, cw.programmers.XMEGAProgrammer, "../hardware/victims/firmware/simpleserial-base/simpleserial-base-CWLITEXMEGA.hex")
#cw.program_target(scope, cw.programmers.STM32FProgrammer, "path/to/firmware.hex")
#cw.program_target(scope, cw.programmers.AVRProgrammer, "path/to/firmware.hex")

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2095 bytes


2. For other targets, you'll need to use an external programmer or a debugger to flash the firmware onto the target. 

Whatever your case, upload the firmware we built earlier to the target device. Next we'll be learning how to capture power traces and communicate with the target.

## Communication with the Target

Communication with targets, which is done through the `SimpleSerial target` object we got earlier, is grouped into two categories:

1. Raw serial via `target.read()`, `target.write()`, `target.flush()`, etc. 

1. SimpleSerial commands via `target.simpleserial_read()`, `target.simpleserial_write()`, `target.simpleserial_wait_ack()`, etc.

The firmware we uploaded uses the simpleserial protocol (https://wiki.newae.com/SimpleSerial), so we'll start off with simpleserial. Later, we'll use the raw serial commands to send the same messages.

If you check the simpleserial-base firmware (`simpleserial-base.c`) you'll find that for the simpleserial `'p'` command, the target will echo back the command. Let's try that out now:

In [7]:
msg = bytearray([0]*16) #simpleserial uses bytearrays
target.simpleserial_write('p', msg)

Let's check if we got a response:

In [8]:
print(target.simpleserial_read('r', 16))

CWbytearray(b'00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00')


It also has a `'k'` command. Try sending that now:

In [9]:
target.simpleserial_write('k', msg)

That command doesn't return anything to us, but it should ack and give us an error return:

In [10]:
print(target.simpleserial_wait_ack()) #should return 0

0


Simpleserial messages generally take the form:

```python
command_character + ascii_encoded_bytes + '\n'
```

For our first command, `command_character='p'` and `ascii_encoded_bytes="00"*16` (keep in mind this isn't a binary `0x00`, it's ASCII `"00"`, which has a binary value of `0x3030`). Try resending the `'p'` command from earlier using `target.write()`:

In [11]:
target.write('p' + "ff"*16 + "\n") #fill in the rest here

A simple `target.read()` will return all the characters that have been sent back from the target so far. Let's see what the device returned to us:

In [12]:
recv_msg = ""

In [13]:
recv_msg += target.read() #you might have to run this block a few times to get the full message
print(recv_msg)

rFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
z00



The simpleserial commands are usually sufficient for taking to simpleserial firmware, but you'll need the raw serial commands for some of the other labs.

## SimpleSerial 2

As of ChipWhisperer 5.4, a new target communication protocol, SimpleSerial 2 is available. It has a number of advantages over the old SimpleSerial protocol:

1. Data is unencoded binary (except for byte stuffing) instead of ACII encoded, meaning far more data can be sent per packet (just under double)
1. It's got a command and subcommand field that get passed to the target callback functions, meaning more can be done per packet. For example, the simpleserial-aes firmware has been modified to allow plaintext, key, and/or mask to be set with a single packet.
1. It's got a length field, meaning the same target commands can take different length packets depending on the situation. This, for example, allows both plaintext and key (or only one) to be send with the same command in simpleserial-aes.
1. It's got an 8-bit CRC (0xA6) for data integrety.
1. Frames are consistant overhead byte stuffed (COBS). This makes it easy to reset communication (done simply by sending two 0x00 bytes) and helps catch malformed length bytes.

It is, however, a more complicated protocol. You cause use it by compiling firmware with `SS_VER=SS_VER_2_0` and using `cw.targets.SimpleSerial2`. In `sca101` and `fault101`, it can be used by setting `SS_VER='SS_VER_2_0'` when setting `PLATFORM` and `SCOPETYPE`.

## Capturing Traces

Now that the target's programmed and we know how to communicate with it, let's start recording some power traces! To capture a trace:

1. Arm the ChipWhisperer with `scope.arm()`. It will begin capturing as soon as it is triggered (which in our case is a rising edge on `gpio4`).
1. `scope.capture()` will read back the captured power trace, blocking until either ChipWhisperer is done recording, or the scope times out. Note that the error return will tell you whether or not the scope timed out. It does not return the captured scope data.
1. You can read back the captured power trace with `scope.get_last_trace()`.

`simpleserial_base` will trigger the ChipWhisperer when we send the `'p'` command. Try capturing a trace now:

In [14]:
msg = bytearray([255,21,12,12,211,11,35,67,90,10,12,13,14,0,56,27])
print(msg)

CWbytearray(b'ff 15 0c 0c d3 0b 23 43 5a 0a 0c 0d 0e 00 38 1b')


In [15]:
scope.arm()
target.simpleserial_write('p', msg)
scope.capture()
scope.get_last_trace()

array([ 0.06738281,  0.11816406,  0.12597656, ..., -0.02636719,
        0.09570312,  0.16503906])

ChipWhisperer also has a `capture_trace()` convience function that:

1. Optionally sends the `'k'` command
1. Arms the scope
1. Sends the `'p'` command
1. Captures the trace
1. Reads the return `'r'` message
1. Returns a `Trace` class that groups the trace data, `'p'` message, the `'r'` message, and the `'k'` message

It isn't always the best option to use, but it's usually sufficient for most simpleserial applications

In [16]:
cw.capture_trace(scope=scope, target=target, plaintext=msg)

Trace(wave=array([ 0.06738281,  0.11425781,  0.12890625, ..., -0.03027344,
        0.09960938,  0.16601562]), textin=CWbytearray(b'ff 15 0c 0c d3 0b 23 43 5a 0a 0c 0d 0e 00 38 1b'), textout=CWbytearray(b'ff 15 0c 0c d3 0b 23 43 5a 0a 0c 0d 0e 00 38 1b'), key=None)

## Conclusion

And that's it! You should be all ready to continue on to SCA101 (or any other of our courses).

We've glossed over some stuff here, so consult the [API documentation](https://chipwhisperer.readthedocs.io/en/latest/api.html) or ask on our [forums](https://forum.newae.com) if you get stuck.

As a final step, we should disconnect from the hardware so it doesn't stay "in use" by this notebook.

In [17]:
scope.dis()
target.dis()